# Parcel Logistics

## Table of Contents

1. [Introduction](#introduction)
1. [Mathematical Model](#model)
    1. [Parameters](#parameters)
    1. [Decision Variables](#decision_variables)
    1. [Constraints](#constraints)
    1. [Objective Function](#objective) 
1. [Dataset and Format](#data)
    1. [Dataset](#dataset)
    1. [Dataset Format](#dataset_format)
1. [Implementation](#implementation)
1. [Conclusion](#conclustion)

<a name="introduction"></a>
### 1. Introduction ###


<img src="introduction-to-vrp.svg" style="width: 50%; display: block; margin: 0 auto;" title="Image source: https://pyvrp.org/setup/introduction_to_vrp.html">


Vehicle Routing Problems (VRPs) are a class of combinatorial optimization problems
that involve the optimization of vehicle routes to efficiently serve a set of customers.
These problems arise in various real-world applications, including logistics, transportation, and supply chain management.

The simplest form of VRP is the Capacitated Vehicle Routing Problem (CVRP), where a fleet of vehicles with limited capacity must be dispatched from a depot to serve a set of customers with known demands. The objective is to minimize the total distance traveled by the vehicles while ensuring that all customer demands are satisfied and that no vehicle exceeds its capacity.

Several variants of the VRP have been studied in the literature, each tailored to specific real-world scenarios. Some of these variants include:

* Vehicle Routing Problem with Time Windows (VRPTW): This variant introduces time constraints for each customer, requiring vehicles to arrive within specified time windows.
* Vehicle Routing Problem with Pickup and Delivery (VRPPD): In this variant, each customer has both a pickup and a delivery location, and the vehicle must pick up items at the pickup location before delivering them to the delivery location.
* Vehicle Routing Problem with Simultaneous Pickup and Delivery (VRPSPD): Similar to VRPPD, but the pickup and delivery of each customer must be performed simultaneously.
* Vehicle Routing Problem with Backhauls (VRPB): This variant involves two types of customers: linehaul customers (deliveries) and backhaul customers (pickups). The objective is to minimize the total distance traveled while satisfying all customer demands.
* Capacitated Vehicle Routing Problem (CVRP): In this variant, a fleet of vehicles with limited capacity must be dispatched from a depot to serve a set of customers with known demands.
The objective is to minimize the total distance traveled by the vehicles while ensuring that all customer demands are satisfied and that no vehicle exceeds its capacity.
 
This project focuses on the Capacitated Vehicle Routing Problem (CVRP).
CVRP is a challenging problem due to its combinatorial nature and the need to balance multiple conflicting objectives.
The problem complexity grows exponentially with the number of vehicles and customers, making it difficult to find optimal solutions,
especially for large-scale instances.

Traditional approaches to solving CVRP include exact methods,
such as branch-and-bound and cutting-plane algorithms, and heuristic and metaheuristic methods,
such as local search, tabu search, simulated annealing, and genetic algorithms.
While exact methods can guarantee optimal solutions, they are often computationally expensive and impractical for large-scale instances.
Heuristic and metaheuristic methods, on the other hand, provide approximate solutions in a more efficient manner.

In recent years, there has been a growing interest in using mathematical programming techniques to solve CVRP.
Mathematical programming formulations, such as integer linear programming (ILP) and mixed-integer linear programming (MILP),
can be used to model the problem and obtain optimal or near-optimal solutions.
However, solving large-scale CVRP instances using exact methods can be computationally intensive.

This report presents a novel approach to solving CVRP using linear programming.
By reformulating the problem as a linear program, we aim to develop a more efficient and scalable solution method.
The linear programming formulation will be based on a relaxation of the original CVRP problem,
and we will explore techniques to strengthen the formulation and improve the quality of the solutions.

<a name="model"></a>
### 2. Mathematical Model ###

In the following, we consider a complete graph *G = (V, E)*,
where *V* is the vertex set and *E* is the edge set.
The vertex set *V* is partitioned into *V = V<sub>d</sub> U V<sub>c</sub>*,
where *V<sub>d</sub> = {0, 1, 2, ..., m-1}* represent the set of *m*depots,
and *V<sub>c</sub> = {m, m+1, ... m+n}* denotes the set of *n* clients.
Each edge *(i, j) &#8712; E* has a weight *d<sub>i,j</sub>* denoting the travelling cost (e.g., distance)
when going from *i &#8712; V* to *j &#8712; V*.
A fleet of vehicles *K<sub>i</sub>* is assumed to be available at each depot *i &#8712; V<sub>d</sub>*.

In this project, we study the capacitated vehicle routing problem (CVRP).
In CVRP, each client *i &#8712; V<sub>c</sub>* has a demand *q<sub>i</sub> >= 0*,
and there is a single depot, *V<sub>d</sub> = {0}*.
It is also assumed that the fleet of vehicles is homogeneous,
meaning that they all have the same maximum capacity *Q*.

A feasible solution to the CVRP consists of a set of routes that all begin and end at the depot,
such that each client is visited exactly once and none of the routes exceeds the vehicle capacity.
The objective is to find a feasible solution that minimizes the total travelling cost.

We formualte the CVRP problem as a linear programming optimization problem.
In the following, we will define the three key components, decision variables, constraints, and objective function.
Before the definitions, we will introduce the parameters used in the problem first.

<a name="parameters"></a>
#### 2.A. Parameters ####
* *d<sub>i,j</sub>* : the distance between the client *i* and the client *j*.
* *q<sub>i</sub>* : the demand for the clinet *i*.

<a name="decision_variables"></a>
#### 2.B. Decision Variables ####
* *x<sub>i,j,k</sub>* : a binary variable denoting the edge *(i, j) &#8712; E* from the client *i* to the client *j* is visited by the vehicle *k*
* *y<sub>i,k</sub>* : a binary variable denoting the demand from the client *i* is picked by the vehicle *k*.

<a name="constraints"></a>
#### 2.C. Constraints ####

* Constraint 1 : Each client *i &#8712; V* is visited once. That is, there is an active edge starting from the client *i* and an active arriving on the client *i*. We model this constraint as *∑<sub>k</sub> ∑<sub>j</sub> x<sub>i,j,k</sub> = ∑<sub>k</sub> ∑<sub>j</sub> x<sub>xj,i,k</sub> = 1* for all *i &#8712; V &#8726; {0}*

* Constraint 2 : If an active edge going into the client *i* or going out of the client *i* is visited by the vehicle *k*, the demand *q* of the client *i* is assigned to the vehicle *k*. We model this constraint as *∑<sub>j</sub> x<sub>i,j,k</sub> = ∑<sub>j</sub> x<sub>j,i,k</sub> = y<sub>i,k</sub>* for all *i &#8712; V &#8726; {0}* and all *k &#8712; K*

* Constraint 3 : The total demands picked up by a vehicle can not exceed its capacity *Q*. Here, we assume all vehicles have the same *Q*. We model this constraint as *∑<sub>i</sub> q<sub>i</sub> * y<sub>i,k</sub> <= Q* for all *k  &#8712; K*

* Constraint 4 : Exactly *K* clients starting with the depot and arriving at the depot. That is, there are *K* active edges going out of the depot and *K* active edges going into the depot. We model this constraint as *∑<sub>k</sub> ∑<sub>j</sub> x<sub>0,j,k</sub> = ∑<sub>k</sub> ∑<sub>i</sub> x<sub>i,0,k</sub> = |K|*  

* Constraint 5 : There are no subtours. We model this constraint as *∑<sub>i</sub> ∑<sub>j ∉ S</sub> x<sub>i,j,k</sub> >= y<sub>h,k</sub>* for *S ⊆ V &#8726; {0}* and *h &#8712; S* and *k &#8712; K* and

* Constraint 6 : *x<sub>i,j,k</sub>* is a binary variable. We model this constraint as *x<sub>i,j,k</sub> &#8712; {0,1}* for all *i &#8712; V* and all *j &#8712; V* and all *k &#8712; K*

* Constraint 7 : *y<sub>i,k</sub>* is a binary variable. We model this constaint as *y<sub>i,k</sub> &#8712; {0,1}* for all *i &#8712; V* and all *k &#8712; K* 

<a name="objective"></a>
#### 2.D. Objective Function ####
The objective function of CVPR is to minimize the total travelling cost, that is,
*minimize ∑<sub>k</sub> ∑<sub>i</sub> ∑<sub>j</sub>d<sub>i,j</sub>x<sub>i,j,k</sub>* for all *k &#8712; K*, *i &#8712; V*, and *j &#8712; V*. 

<a name="data"></a>
### 3. Dataset and Format ###

<a name="dataset"></a>
#### 3.A. Dataset ####

The dataset we are using in this project is from the webpage, https://pyvrp.org/examples/quick_tutorial.html


<a name="dataset_format"></a>
#### 3.B. Dataset Format ####
The dataset includes two text files. The first text file, named data1_location.txt, records the coordinates of the depot and the clients. The first entry is the coordinate of the depot and the subsequent entries denote the coordinates of all the clients. The second text file, named data1_demand.txt, records the demands of the depot and the clients. The first entry denotes the demand for the depot, which is 0, as the depot has no demand for vehciles to pick up from. The subsequent entries denote the demands of all the clients. 

<a name="implementation"></a>
### 4. Implementation ###

In [6]:
# Install Ipopt
using Pkg
Pkg.add("Ipopt")
Pkg.add("PyPlot")
Pkg.add("PyCall")   
 
# use PyCall
using PyCall        
 
# import python library
#@pyimport numpy      

using JuMP, Ipopt
using PyPlot
using CSV

#import numpy as np
#import matplotlib.pyplot as plt
#from scipy.spatial import distance
#import random
#import pulp
#import ortools 

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [ ]:
function CVRP(clients, vehicles, capacity, demands, costs):
    m = Model(Ipopt.Optimizer)

    # ------ Decsion variables ------

    # Define x[i,j,k]
    @variable(x[clients+2, clients+2, vehicles])

    # Define y[i,k]
    @variable(y[clients+2, vehciles])

    
    # ------ Constraints ------
    # Constraint 1 : Each client is visited once, ∑k ∑j xi,j,k = ∑k ∑j xj,i,k = 1 for all i ∈ V ∖ {0}
    for i in 2:clients+1
        @constraint(m, sum(x[i,j,k] for k in 1:vehicles, for j in 2:clients) == 1)
        @constraint(m, sum(x[j,i,k] for k in 1:vehicles, for j in 2:clients) == 1)
    end
    
    # Constraint 2 : ∑j xi,j,k = ∑j xj,i,k = yi,k for all i ∈ V ∖ {0} and all k ∈ K
    for i in 2:clients+1
        for k in 1:vehicles
            @constraint(m, sum(x[i,j,k] for j in 2:clients+1) == y[i,k])
            @constraint(m, sum(x[j,i,k] for j in 2:clients+1) == y[i,k])
        end
    end

    
    # Constraint 3 : the total demands picked up by a vehicle can not exceed the maximum capacity

    
    # Constriant 6
    @constraint(0<= x[i,j] <= 1 for i in 1:clients, j in 1:clients)

    # Constraint 7
    @constraint(0<= y[i] for i in 1:clients+1)
    
    # ------ Objective function ------

    
end

In [ ]:
# Running data1

# Define the number of clients
clients = 16

# Define the number of vehicles
vehicles = 4

# Define the maximum capacity of a vehicle
capacity = 15

# Read in the demands of the clients
demands = []
open("data1_demand.txt") do file
    for line in eachline(file)
        push!(demands, line)
    end
end

# Read in the locations of the depot and the clients
locations = []
open("data1_location.txt") do file
    for line in eachline(file)
        coordinates = parse.(Int, split(line, ","))
        push!(locations, coordinates)
    end
end

# Define the cost matrix based on the read-in locations
costs = zeros(clients+2, clients+2)
for i in 1:clients+1
    for j in 1:clients+1
        costs[i][j] = (locations[i][0] - locations[j][0])^2 + (locations[i][1] - locations[j][1])^2
    end
end

for i in 1:clients+1
    costs[i][clients+2] = costs[i][1]
end

for i in 1:clients+1
    costs[clients+2][i] = costs[1][i]
end


routes = CVRP(clients, vehicles, capacity, demands, costs)

<a name="conclusion"></a>
### 5. Conclusion ###